<a href="https://colab.research.google.com/github/AnirudhSreeram/Machine-Learning-codes/blob/master/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [0]:
## Download the training data set from torchvision CIFAR10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True,
                                        transform=transform
                                        )

In [0]:
##Create a data loader object to handel batching
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [0]:
## Download the testing data set from torchvision CIFAR10
testset =torchvision.datasets.CIFAR10(root='./data' , 
                                      train=False,
                                      download=True,
                                      transform=transform)

In [0]:
##Create a data loader object to handel batching
testloader=torch.utils.data.DataLoader(testset,batch_size=4, shuffle=True, num_workers=2)

In [0]:
###print the class 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
#Set the learining parameters for the model
learning = {'rate' : 0.0001,
            'minEpoch' : 5,
            'lrScale' : 0.5,
            'batchSize' : 128,
            'lrScaleCount' : 18,
            'spliceSize' : 21,
            'minValError' : 0.002}

In [0]:
#create a class to train the model with a fit function
class Train():
     def __init__(self):
      self.losscv_previous_1=10.0
      self.losscv_previous_2=0.0
      self.losscv_current=0.0

     def fit (self, net, trainloader, testloader,criterion,optimizer,epoch,totalepoch):
        net.train()
        print('epoch = %d/%d'%(epoch+1,totalepoch))
        running_loss_tr =0.0
        correct_tr = 0.0
        total_tr = 0.0
        for i, data in enumerate(trainloader,0):
         inputs, labels =data
         labels=labels.cuda()
         inputs = Variable(inputs)
         optimizer.zero_grad()
         outputs =net(inputs.cuda().float())
         loss =criterion(outputs, labels.detach())
         loss.backward()
         optimizer.step()
         running_loss_tr += loss.item()
         _,predicted =torch.max(outputs.data,1)
         total_tr += labels.size(0)
         correct_tr += (predicted == labels).sum().item()
         if i % 2000 == 1999:
          print ('[%d,%5d] loss_tr: %.3f' % (epoch+1,i+1,running_loss_tr/2000))
          running_loss_tr =0.0
          print ('ACCURACY_TR  : %.3f %%' % (100 * correct_tr / total_tr))
          #break
        print ('Finished Training')
#corss validation step   
        correct_cv = 0.0
        total_cv = 0.0
        net.eval()
        running_loss_cv=0.0
        for j, data in enumerate(testloader,0):
           images,label=data
           label=label.cuda()
           images = Variable(images)
           output=net(images.cuda().float())
           loss_cv =criterion(output, label.detach())
           running_loss_cv += loss_cv.item()
           total_cv += label.size(0)
           _,predicted =torch.max(output.data,1)
           correct_cv += (predicted == label).sum().item()
           if j % 200 == 199 :
            print ('[%d,%5d] loss_cv: %.3f' % (epoch+1,j+1,running_loss_cv/200))
            self.losscv_current=running_loss_cv/200
            running_loss_cv =0.0
            print ('ACCURACY_CV  : %.3f %%' % (100 * correct_cv / total_cv))
            path = F"/content/gdrive/My Drive/CIFAR10.model"
            torch.save(net.state_dict(),path)


In [0]:
#Initialize the Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 8, 3)
        self.fc1 = nn.Linear(8 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 8 * 6 *6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [0]:
#Mount the Drive to save the model
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Begin the training
net=Net()
net=net.cuda()
print(net)
train1=Train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning['rate'],amsgrad=True)

In [0]:
path = F"/content/gdrive/My Drive/CIFAR10.model"
for epoch in range(learning['minEpoch']-1):
 if(epoch >= 1):
         net.load_state_dict(torch.load(path))
 t0 = time.time()
 abc=train1.fit(net, trainloader, testloader,criterion,optimizer,epoch,4)
 print('{} seconds'.format(time.time() - t0))
